In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm
from PIL import Image
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms as T
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from timeit import default_timer as timer

In [2]:
!git clone https://github.com/fbcotter/pytorch_wavelets

Cloning into 'pytorch_wavelets'...
remote: Enumerating objects: 978, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 978 (delta 79), reused 92 (delta 46), pack-reused 836 (from 1)
Receiving objects: 100% (978/978), 6.80 MiB | 26.48 MiB/s, done.
Resolving deltas: 100% (663/663), done.


In [3]:
%cd pytorch_wavelets

/kaggle/working/pytorch_wavelets


In [4]:
pip install .

Processing /kaggle/working/pytorch_wavelets
  Preparing metadata (setup.py) ... done
  Created wheel for pytorch_wavelets: filename=pytorch_wavelets-1.3.0-py3-none-any.whl size=54852 sha256=a152dca16c719a8efc6597101c877b8fb89f125277a56c551f32f258d5365c6f
  Stored in directory: /tmp/pip-ephem-wheel-cache-cth5pa55/wheels/b1/4f/c7/1300c122fca795cefecfec6ebfe7f3277428f044a644235bb1
Successfully built pytorch_wavelets
Note: you may need to restart the kernel to use updated packages.


In [5]:
!nvidia-smi

Thu Dec  5 19:25:43 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             25W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
NUM_EPOCHS = 100
BATCH_SIZE = 640
TRAIN_FOLDER_DIR = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train"
TEST_FOLDER_DIR = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test"
LEARNING_RATE = 5e-3
NUM_WORKERS = os.cpu_count()
PIN_MEMORY = True
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
class RealFakeDataset(Dataset):
    def __init__(self, folder_dir, transform=None):
        super().__init__()
        self.transform = transform
        self.real_img_dir = os.path.join(folder_dir, "REAL")
        self.fake_img_dir = os.path.join(folder_dir, "FAKE")
        self.data = []
        for pth in os.listdir(self.real_img_dir):
            self.data.append((os.path.join(self.real_img_dir, pth), 1))

        for pth in os.listdir(self.fake_img_dir):
            self.data.append((os.path.join(self.fake_img_dir, pth), 0))

        np.random.shuffle(self.data)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        img_pth, label = self.data[item]

        img = np.array(Image.open(img_pth).convert("RGB"), dtype=np.float32) / 255
        if self.transform:
            img = self.transform(img)

        return img, torch.tensor([label], dtype=torch.float32)

In [8]:
transform = T.Compose([
    T.ToTensor(),
    T.RandomHorizontalFlip(p=0.25),
    T.RandomVerticalFlip(p=0.25),
    T.RandomRotation(degrees=30),
])

train_data = RealFakeDataset(folder_dir=TRAIN_FOLDER_DIR, transform=transform)
test_data = RealFakeDataset(folder_dir=TEST_FOLDER_DIR, transform=transform)

train_dataloader = DataLoader(train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              num_workers=NUM_WORKERS,
                              pin_memory=PIN_MEMORY
                              )

test_dataloader = DataLoader(test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False,
                             num_workers=NUM_WORKERS,
                             pin_memory=PIN_MEMORY
                             )

In [9]:
len(train_data), len(test_data)

(100000, 20000)

In [10]:
def HFRIBlock(x):
    """
    High Frequency Representation of Image
    """
    freq_represnetation = torch.fft.fft2(x, dim=(-2, -1))
    freq_represnetation[..., :x.shape[-2] // 4, :] = 0
    freq_represnetation[..., :, :x.shape[-1] // 4] = 0
    processed_image = torch.fft.ifft2(freq_represnetation, dim=(-2, -1)).real
    return processed_image


class ConvLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1, stride=1):
        super(ConvLayer, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding, stride=stride, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )


    def forward(self, x):
        return self.conv(x)


class HFRFSBlock(nn.Module):
    """
    High Frequency Representation of features across spatial
    """
    def __init__(self, in_channels, out_channels):
        super(HFRFSBlock, self).__init__()
        self.conv = ConvLayer(in_channels, out_channels)

    def forward(self, x):
        spatial_freq = torch.fft.fft2(x, dim=(-2, -1))
        spatial_freq[..., :x.shape[-2] // 4, :] = 0
        spatial_freq[..., :, :x.shape[-1] // 4] = 0
        spatial_processed = torch.fft.ifft2(spatial_freq, dim=(-2, -1)).real

        return self.conv(spatial_processed)


class HFRFCBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(HFRFCBlock, self).__init__()
        self.conv = ConvLayer(in_channels, out_channels)

    def forward(self, x):
        channel_freq = torch.fft.fft(x, dim=1)
        channel_freq[:, :x.shape[1] // 4, :, :] = 0
        channel_processed = torch.fft.ifft(channel_freq, dim=1).real

        return self.conv(channel_processed)


class FCLBlock(nn.Module):
    """
    Frequency Convolution Layer
    """
    def __init__(self, in_channels, out_channels):
        super(FCLBlock, self).__init__()
        self.residual_conv_start = ConvLayer(in_channels, out_channels)
        self.phase_conv = ConvLayer(out_channels, out_channels)
        self.amplitude_conv = ConvLayer(out_channels, out_channels)
        self.residual_conv_final = ConvLayer(out_channels, out_channels)

    def forward(self, x):
        x = self.residual_conv_start(x)
        freq_represnetation = torch.fft.fft2(x, dim=(-2, -1))
        amplitude = freq_represnetation.abs()
        phase = torch.angle(freq_represnetation)

        amp_processed = self.amplitude_conv(amplitude)
        phase_processed = self.phase_conv(phase)

        combined_freq = torch.polar(amp_processed, phase_processed)

        processed = torch.fft.ifft2(combined_freq, dim=(-2, -1)).real

        return self.residual_conv_final(processed)

class FreqNet(nn.Module):
    def __init__(self):
        super(FreqNet, self).__init__()
        # self.block1 = nn.Sequential(
        #     HFRFCBlock(3, 3),
        #     FCLBlock(3, 32)
        # )
        # self.block2 = nn.Sequential(
        #     HFRFSBlock(32, 32),
        #     HFRFCBlock(32, 32),
        #     FCLBlock(32, 64),
        #     ConvLayer(64, 64)
        # )
        self.block3 = nn.Sequential(
            HFRFSBlock(3, 3),
            FCLBlock(3, 64),
            HFRFSBlock(64, 64),
            FCLBlock(64, 128),
            ConvLayer(128, 128, stride=2)
        )
        self.final_conv = ConvLayer(128, 256, stride=2)
        # self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        # self.fc = nn.Linear(in_features=128, out_features=1, bias=True)

    def forward(self, x):
        hfri_out = HFRIBlock(x)
        # out = self.block3(self.block2(self.block1(hfri_out)))
        out = self.final_conv(self.block3(hfri_out))
        # pool = self.avgpool(out)
        # return self.fc(pool.reshape(x.shape[0], -1))
        return out

In [11]:
from pytorch_wavelets import DWT2D

class DWTLayer(nn.Module):
    def __init__(self, wavelet_):
        super(DWTLayer, self).__init__()
        self.dwt = DWT2D(J=1, wave=wavelet_)

    def forward(self, x):
        num_channels = x.shape[1]
        ll_lis, lh_lis, hl_lis = [], [], []
        for i in range(num_channels):
            channel = x[:, i, :, :].unsqueeze(1)
            low, high = self.dwt(channel)
            lh, hl, hh = torch.unbind(high[0], dim=2)
            ll_lis.append(low)
            lh_lis.append(lh)
            hl_lis.append(hl)

        ll = torch.stack(ll_lis, dim=1).squeeze(dim=2)
        hl = torch.stack(hl_lis, dim=1).squeeze(dim=2)
        lh = torch.stack(lh_lis, dim=1).squeeze(dim=2)

        return ll, lh, hl


class WaveletAttentionBlock(nn.Module):
    def __init__(self, wavelet='haar'):
        super(WaveletAttentionBlock, self).__init__()
        self.dwt = DWTLayer(wavelet)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        ll, lh, hl = self.dwt(x)
        out = (ll * self.softmax(lh + hl)) + ll
        return out

class WaveletAttentionStride(nn.Module):
    def __init__(self, in_features, out_features, wavelet='haar'):
        super(WaveletAttentionStride, self).__init__()
        self.att_block = WaveletAttentionBlock(wavelet)
        self.conv = nn.Conv2d(in_features, out_features, kernel_size=1, stride=1)

    def forward(self, x):
        out = self.att_block(x)
        return self.conv(out)

In [12]:
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, identity_downsample=None, stride=1):
        super(Block, self).__init__()

        self.expansion = 4
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(out_channels*self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample

    def forward(self, x):
        identity = x
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x


class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        # self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.wave_attn_pool = WaveletAttentionStride(64, 64)
        self.layer1 = self._make_layer(
            block, layers[0], out_channels=64, stride=1
        )
        self.layer2 = self._make_layer(
            block, layers[1], out_channels=128, stride=2
        )
        self.layer2_wave = WaveletAttentionStride(256, 512)
        self.layer3 = self._make_layer(
            block, layers[2], out_channels=256, stride=2
        )
        self.layer4 = self._make_layer(
            block, layers[3], out_channels=512, stride=2
        )
        self.freqnet = FreqNet()

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512*4, num_classes)

    def forward(self, x):
        freq_x = self.freqnet(x)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        # x = self.maxpool(x)
        x = self.wave_attn_pool(x)
        x = self.layer1(x) + freq_x
        x = self.layer2(x) + self.layer2_wave(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x

    def _make_layer(self, block, num_residual_blocks, out_channels, stride):
        identity_downsample = None
        layers = []
        if stride != 1 or self.in_channels != out_channels*4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels*4, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels*4)
            )
        layers.append(Block(self.in_channels, out_channels, identity_downsample, stride))
        self.in_channels = out_channels*4

        for i in range(num_residual_blocks -1):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)


def ResNet50(img_channels=3, num_classes=1):
    return ResNet(Block, [3, 4, 6, 3], image_channels=img_channels, num_classes=num_classes)

In [13]:
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving Checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint, model):
    print("==> Loading Checkpoint")
    model.load_state_dict(checkpoint["state_dict"])

In [14]:
def train_step(model,
               loader,
               loss_fn,
               optimizer,
               device=DEVICE,
               ):

    model.train()
    train_loss, train_acc = 0, 0
    loop = tqdm(loader)
    for batch, (X, y) in enumerate(loop):
        X, y = X.to(device), y.to(device)
        model = model.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        y_pred_class = torch.round(torch.sigmoid(y_pred))
        train_acc += (y_pred_class == y).sum().item() / len(y_pred)
        loop.set_postfix(loss=loss.item())

    train_loss = train_loss / len(loader)
    train_acc = train_acc / len(loader)

    return train_loss, train_acc


def test_step(model,
              loader,
              loss_fn,
              device=DEVICE):
    model.eval()
    test_loss, test_acc = 0, 0
    with torch.inference_mode():
        for batch, (X, y) in enumerate(loader):
            X, y = X.to(device), y.to(device)
            model.to(device)
            test_pred = model(X)
            loss = loss_fn(test_pred, y)
            test_loss += loss.item()
            test_pred_class = torch.round(torch.sigmoid(test_pred))
            test_acc += (test_pred_class == y).sum().item() / len(test_pred)

    test_loss = test_loss / len(loader)
    test_acc = test_acc / len(loader)

    return test_loss, test_acc


def train(model: torch.nn.Module,
          train_dataloader_: torch.utils.data.DataLoader,
          test_dataloader_: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int = NUM_EPOCHS,
          device=DEVICE,
          patience=5):

    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []}
    scheduler = ReduceLROnPlateau(optimizer, patience=patience)
    for epoch in range(epochs):
        train_loss, train_acc = train_step(model=model,
                                           loader=train_dataloader_,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer,
                                           device=device)

        test_loss, test_acc = test_step(model=model,
                                        loader=test_dataloader_,
                                        loss_fn=loss_fn,
                                        device=device)
        scheduler.step(test_loss)

        print(f"Epoch: {epoch+1} | Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f} | Test loss: {test_loss:.4f} | Test acc: {test_acc:.4f}")

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    return results

In [15]:
model = ResNet50()
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(pytorch_total_params)
print(model)

24782552
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (wave_attn_pool): WaveletAttentionStride(
    (att_block): WaveletAttentionBlock(
      (dwt): DWTLayer(
        (dwt): DWTForward()
      )
      (softmax): Softmax(dim=1)
    )
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
  )
  (layer1): Sequential(
    (0): Block(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
 

In [16]:
%cd ..

/kaggle/working


In [17]:
def trainer(model):
    # model = CustomCNN(0.3, 0.3).to(DEVICE)
    model.to(DEVICE)
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(params=model.parameters(),
                                 lr=LEARNING_RATE)
    start_time = timer()
    train(model, train_dataloader, test_dataloader, optimizer, loss_fn, epochs=NUM_EPOCHS, device=DEVICE, patience=5)
    end_time = timer()

    print(f"Total Training Time: {end_time-start_time:.3f}")
    checkpoint = {
    "state_dict": model.state_dict(),
    "optimizer": optimizer.state_dict()
    }
    save_checkpoint(checkpoint, filename="res50_freq_checkpoint.pth.tar")

In [18]:
trainer(model)

100%|██████████| 157/157 [04:12<00:00,  1.61s/it, loss=0.437]


Epoch: 1 | Train loss: 0.5841 | Train acc: 0.7087 | Test loss: 0.6933 | Test acc: 0.6696


100%|██████████| 157/157 [03:01<00:00,  1.16s/it, loss=0.334]


Epoch: 2 | Train loss: 0.3020 | Train acc: 0.8704 | Test loss: 0.2928 | Test acc: 0.8744


100%|██████████| 157/157 [03:01<00:00,  1.15s/it, loss=0.194]


...

Epoch: 98 | Train loss: 0.0695 | Train acc: 0.9739 | Test loss: 0.1111 | Test acc: 0.9590


100%|██████████| 157/157 [02:58<00:00,  1.14s/it, loss=0.0544]


Epoch: 99 | Train loss: 0.0702 | Train acc: 0.9736 | Test loss: 0.1096 | Test acc: 0.9601


100%|██████████| 157/157 [02:58<00:00,  1.14s/it, loss=0.078]


Epoch: 100 | Train loss: 0.0678 | Train acc: 0.9744 | Test loss: 0.1112 | Test acc: 0.9598
Total Training Time: 19929.348
=> Saving Checkpoint


In [19]:
import os
from IPython.display import FileLink

os.chdir(r'/kaggle/working')
FileLink(r'res50_freq_checkpoint.pth.tar')

/kaggle/working/res50_freq_checkpoint.pth.tar